In [1]:
from torchtext import data,datasets
from torchtext.vocab import GloVe,FastText,CharNGram
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import torch
from imdb import IMDB
import sys

In [2]:
torch.__version__
sys.version

'3.6.2 |Anaconda custom (64-bit)| (default, Jul 20 2017, 13:51:32) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]'

In [3]:
sys.getdefaultencoding()

'utf-8'

In [4]:
is_cuda = False

if torch.cuda.is_available():
    is_cuda=True


In [5]:
TEXT = data.Field(lower=True, batch_first=True,fix_length=40,)
LABEL = data.Field(sequential=False,)

In [6]:
train, test = IMDB.splits(TEXT, LABEL)

In [7]:
type(train)

imdb.IMDB

In [8]:
print('train.fields', train.fields)
print('len(train)', len(train))
print('vars(train[0])', vars(train[0]))



train.fields {'text': <torchtext.data.field.Field object at 0x7f6db748ef60>, 'label': <torchtext.data.field.Field object at 0x7f6db7c44630>}
len(train) 25000
vars(train[0]) {'text': ["i'm", 'not', 'tired', 'to', 'say', 'this', 'is', 'one', 'of', 'the', 'best', 'political', 'thrillers', 'ever', 'made.', 'the', 'story', 'takes', 'place', 'in', 'a', 'fictional', 'state,', 'but', 'obviously', 'it', 'deals', 'with', 'the', 'murder', 'of', 'kennedy.', 'a', 'truthful', 'and', 'honest', 'district', 'attorney', '(played', 'by', 'yves', 'montand)', 'does', 'not', 'believe', 'that', 'the', 'murder', 'was', 'planned', 'and', 'executed', 'by', 'the', 'single', 'man', 'daslow', '(=oswald)', 'and', 'though', 'all', 'other', 'officials', 'want', 'to', 'close', 'the', 'case', 'he', 'continuous', 'to', 'investigate', 'with', 'his', 'team.<br', '/><br', '/>the', 'screenplay', 'is', 'written', 'tight', 'and', 'fast', 'and', 'holds', 'the', 'tension', 'till', 'the', 'end.', 'just', 'the', 'part', 'dealing'

In [9]:
TEXT.build_vocab(train, vectors=GloVe(name='6B', dim=300),max_size=10000,min_freq=10)
LABEL.build_vocab(train,)

In [10]:
LABEL.vocab.freqs

Counter({'neg': 12500, 'pos': 12500})

In [11]:
print('train.fields', train.fields)
print('len(train)', len(train))
print('vars(train[0])', vars(train[0]))

train.fields {'text': <torchtext.data.field.Field object at 0x7f6db748ef60>, 'label': <torchtext.data.field.Field object at 0x7f6db7c44630>}
len(train) 25000
vars(train[0]) {'text': ["i'm", 'not', 'tired', 'to', 'say', 'this', 'is', 'one', 'of', 'the', 'best', 'political', 'thrillers', 'ever', 'made.', 'the', 'story', 'takes', 'place', 'in', 'a', 'fictional', 'state,', 'but', 'obviously', 'it', 'deals', 'with', 'the', 'murder', 'of', 'kennedy.', 'a', 'truthful', 'and', 'honest', 'district', 'attorney', '(played', 'by', 'yves', 'montand)', 'does', 'not', 'believe', 'that', 'the', 'murder', 'was', 'planned', 'and', 'executed', 'by', 'the', 'single', 'man', 'daslow', '(=oswald)', 'and', 'though', 'all', 'other', 'officials', 'want', 'to', 'close', 'the', 'case', 'he', 'continuous', 'to', 'investigate', 'with', 'his', 'team.<br', '/><br', '/>the', 'screenplay', 'is', 'written', 'tight', 'and', 'fast', 'and', 'holds', 'the', 'tension', 'till', 'the', 'end.', 'just', 'the', 'part', 'dealing'

In [12]:
d = vars(TEXT.vocab)

In [13]:
d.keys()

dict_keys(['freqs', 'stoi', 'itos', 'vectors'])

In [14]:
TEXT.vocab.vectors




 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
 0.0466  0.2132 -0.0074  ...   0.0091 -0.2099  0.0539
          ...             ⋱             ...          
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
 0.7724 -0.1800  0.2072  ...   0.6736  0.2263 -0.2919
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
[torch.FloatTensor of size 10002x300]

In [15]:
len(TEXT.vocab.stoi)

10002

In [16]:
train_iter, test_iter = data.BucketIterator.splits((train, test), batch_size=32, device=-1)

train_iter.repeat = False
test_iter.repeat = False

In [17]:
class EmbNet(nn.Module):
    def __init__(self,emb_size,hidden_size1,hidden_size2=400):
        super().__init__()
        self.embedding = nn.Embedding(emb_size,hidden_size1)
        self.fc = nn.Linear(hidden_size2,3)
        
    def forward(self,x):
        embeds = self.embedding(x).view(x.size(0),-1)
        out = self.fc(embeds)
        return F.log_softmax(out,dim=-1)
        
        

In [18]:
model = EmbNet(len(TEXT.vocab.stoi),10)
model = model.cuda()

In [19]:

optimizer = optim.Adam(model.parameters(),lr=0.001)

In [20]:
train_iter, test_iter = data.BucketIterator.splits((train, test), batch_size=32, device=-1,shuffle=True)
train_iter.repeat = False
test_iter.repeat = False

In [21]:
def fit(epoch,model,data_loader,phase='training',volatile=False):
    if phase == 'training':
        model.train()
    if phase == 'validation':
        model.eval()
        volatile=True
    running_loss = 0.0
    running_correct = 0
    for batch_idx , batch in enumerate(data_loader):
        text , target = batch.text , batch.label
        if is_cuda:
            text,target = text.cuda(),target.cuda()
        
        if phase == 'training':
            optimizer.zero_grad()
        output = model(text)
        loss = F.nll_loss(output,target)
        
        running_loss += F.nll_loss(output,target,size_average=False).data[0]
        preds = output.data.max(dim=1,keepdim=True)[1]
        running_correct += preds.eq(target.data.view_as(preds)).cpu().sum()
        if phase == 'training':
            loss.backward()
            optimizer.step()
    
    loss = running_loss/len(data_loader.dataset)
    accuracy = 100. * running_correct/len(data_loader.dataset)
    
    print(f'{phase} loss is {loss:{5}.{2}} and {phase} accuracy is {running_correct}/{len(data_loader.dataset)}{accuracy:{10}.{4}}')
    return loss,accuracy

In [22]:
train_losses , train_accuracy = [],[]
val_losses , val_accuracy = [],[]

In [23]:
train_losses , train_accuracy = [],[]
val_losses , val_accuracy = [],[]

for epoch in range(1,10):

    epoch_loss, epoch_accuracy = fit(epoch,model,train_iter,phase='training')
    val_epoch_loss , val_epoch_accuracy = fit(epoch,model,test_iter,phase='validation')
    train_losses.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)

/home/vishnu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration
  if __name__ == '__main__':


training loss is  0.74 and training accuracy is 12934/25000     51.74
validation loss is   0.7 and validation accuracy is 13327/25000     53.31
training loss is  0.69 and training accuracy is 14178/25000     56.71
validation loss is  0.68 and validation accuracy is 14605/25000     58.42
training loss is  0.64 and training accuracy is 15760/25000     63.04
validation loss is  0.64 and validation accuracy is 15780/25000     63.12
training loss is  0.59 and training accuracy is 16978/25000     67.91
validation loss is  0.62 and validation accuracy is 16583/25000     66.33
training loss is  0.55 and training accuracy is 17956/25000     71.82
validation loss is   0.6 and validation accuracy is 17047/25000     68.19
training loss is  0.51 and training accuracy is 18686/25000     74.74
validation loss is  0.59 and validation accuracy is 17345/25000     69.38
training loss is  0.48 and training accuracy is 19164/25000     76.66
validation loss is  0.59 and validation accuracy is 17596/25000   

## Using pretrained Glove word embeddings

In [24]:
TEXT = data.Field(lower=True, batch_first=True,fix_length=40,)
LABEL = data.Field(sequential=False,)

train, test = IMDB.splits(TEXT, LABEL)

TEXT.build_vocab(train,test, vectors=GloVe(name='6B', dim=300),max_size=10000,min_freq=10)
LABEL.build_vocab(train,)

In [35]:
class EmbNet(nn.Module):
    def __init__(self,emb_size,hidden_size1,hidden_size2=400):
        super().__init__()
        self.embedding = nn.Embedding(emb_size,hidden_size1)
        self.fc1 = nn.Linear(hidden_size2,3)

        
    def forward(self,x):
        embeds = self.embedding(x).view(x.size(0),-1)
        out = self.fc1(embeds)
        return F.log_softmax(out,dim=-1)

In [36]:
model = EmbNet(len(TEXT.vocab.stoi),300,12000)
model = model.cuda()

In [37]:
model.embedding.weight.data = TEXT.vocab.vectors.cuda()

In [38]:
model.embedding.weight.requires_grad = False

In [39]:
#optimizer = optim.SGD(model.parameters(),lr=0.001)
optimizer = optim.Adam([ param for param in model.parameters() if param.requires_grad == True],lr=0.001)

In [40]:
train_iter, test_iter = data.BucketIterator.splits((train, test), batch_size=64, device=-1,shuffle=True)
train_iter.repeat = False
test_iter.repeat = False

In [41]:
def fit(epoch,model,data_loader,phase='training',volatile=False):
    if phase == 'training':
        model.train()
    if phase == 'validation':
        model.eval()
        volatile=True
    running_loss = 0.0
    running_correct = 0
    for batch_idx , batch in enumerate(data_loader):
        text , target = batch.text , batch.label
        if is_cuda:
            text,target = text.cuda(),target.cuda()
        
        if phase == 'training':
            optimizer.zero_grad()
        output = model(text)
        loss = F.nll_loss(output,target)
        
        running_loss += F.nll_loss(output,target,size_average=False).data[0]
        preds = output.data.max(dim=1,keepdim=True)[1]
        running_correct += preds.eq(target.data.view_as(preds)).cpu().sum()
        if phase == 'training':
            loss.backward()
            optimizer.step()
    
    loss = running_loss/len(data_loader.dataset)
    accuracy = 100. * running_correct/len(data_loader.dataset)
    
    print(f'{phase} loss is {loss:{5}.{2}} and {phase} accuracy is {running_correct}/{len(data_loader.dataset)}{accuracy:{10}.{4}}')
    return loss,accuracy

In [42]:
for epoch in range(1,10):

    epoch_loss, epoch_accuracy = fit(epoch,model,train_iter,phase='training')
    val_epoch_loss , val_epoch_accuracy = fit(epoch,model,test_iter,phase='validation')
    train_losses.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)

/home/vishnu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration
  if __name__ == '__main__':


training loss is  0.66 and training accuracy is 15727/25000     62.91
validation loss is  0.64 and validation accuracy is 16489/25000     65.96
training loss is  0.56 and training accuracy is 17809/25000     71.24
validation loss is  0.66 and validation accuracy is 16496/25000     65.98
training loss is  0.53 and training accuracy is 18366/25000     73.46
validation loss is  0.67 and validation accuracy is 16524/25000      66.1
training loss is   0.5 and training accuracy is 18825/25000      75.3
validation loss is   0.7 and validation accuracy is 16416/25000     65.66
training loss is  0.49 and training accuracy is 18990/25000     75.96
validation loss is  0.72 and validation accuracy is 16368/25000     65.47
training loss is  0.48 and training accuracy is 19181/25000     76.72
validation loss is  0.73 and validation accuracy is 16354/25000     65.42
training loss is  0.47 and training accuracy is 19406/25000     77.62
validation loss is  0.75 and validation accuracy is 16231/25000   

In [43]:
for epoch in range(1,10):

    epoch_loss, epoch_accuracy = fit(epoch,model,train_iter,phase='training')
    val_epoch_loss , val_epoch_accuracy = fit(epoch,model,test_iter,phase='validation')
    train_losses.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)

/home/vishnu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration
  if __name__ == '__main__':


training loss is  0.45 and training accuracy is 19673/25000     78.69
validation loss is   0.8 and validation accuracy is 16174/25000      64.7
training loss is  0.45 and training accuracy is 19747/25000     78.99
validation loss is  0.82 and validation accuracy is 16182/25000     64.73
training loss is  0.44 and training accuracy is 19899/25000      79.6
validation loss is  0.83 and validation accuracy is 16145/25000     64.58
training loss is  0.43 and training accuracy is 20005/25000     80.02
validation loss is  0.85 and validation accuracy is 16031/25000     64.12
training loss is  0.42 and training accuracy is 20079/25000     80.32
validation loss is  0.86 and validation accuracy is 16071/25000     64.28
training loss is  0.42 and training accuracy is 20103/25000     80.41
validation loss is  0.87 and validation accuracy is 16063/25000     64.25
training loss is  0.42 and training accuracy is 20107/25000     80.43
validation loss is  0.89 and validation accuracy is 16067/25000   